# Key Features
#### Dynamic NIFTY CSV parsing → avoids ParserError.
#### Dow Jones handled separately via Wikipedia API.
#### Multi-index tagging → symbols in multiple indices are combined with |.
#### Deduplication → keeps one row per (Stock Symbol, Stock Market).
#### Simple, maintainable structure — easy to extend for ISIN, sector, country later.

In [3]:
import pandas as pd
import yfinance as yf
from pathlib import Path
from datetime import datetime, timedelta
import time

# =========================
# Paths
# =========================
INPUT_FILE = Path("../Data/normalized_csv/ALL_INDEX_STOCKS.csv")
OUTPUT_DIR = Path("../Data/stock_ohlcv")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# Read and deduplicate
# =========================
df = pd.read_csv(INPUT_FILE)
df = df.drop_duplicates(subset=["Stock Symbol", "Stock Market"])
print(f"Total unique stocks: {len(df)}")

# =========================
# Function to get Yahoo ticker
# =========================
def get_yahoo_ticker(symbol, market):
    if market.upper() == "NSE":
        return f"{symbol}.NS"
    elif market.upper() == "BSE":
        return f"{symbol}.BO"
    else:
        return symbol

# =========================
# Download OHLCV
# =========================
start_date = datetime.now() - timedelta(days=50*365)
end_date = datetime.now()

for idx, row in df.iterrows():
    symbol = row["Stock Symbol"]
    market = row["Stock Market"]
    name = row["Stock Name"]

    ticker = get_yahoo_ticker(symbol, market)
    out_file = OUTPUT_DIR / f"{symbol}_{market}.csv"

    try:
        print(f"Downloading {symbol} ({market}) ...")
        # data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=False)

        if data.empty:
            print(f"⚠️ No data for {symbol} ({market})")
            continue
        # Save CSV
        data.to_csv(out_file)
        print(f"✅ Saved → {out_file}")
        time.sleep(1)  # polite delay
    except Exception as e:
        print(f"❌ Failed for {symbol} ({market}): {e}")


Total unique stocks: 31
✅ Saved → ..\Data\stock_ohlcv\A_NYSE.csv
❌ Failed for A (NYSE/NASDAQ): Cannot save file into a non-existent directory: '..\Data\stock_ohlcv\A_NYSE'
✅ Saved → ..\Data\stock_ohlcv\AA_NYSE.csv
✅ Saved → ..\Data\stock_ohlcv\AAP_NYSE.csv
✅ Saved → ..\Data\stock_ohlcv\AAPL_NASDAQ.csv
❌ Failed for AAPL (NYSE/NASDAQ): Cannot save file into a non-existent directory: '..\Data\stock_ohlcv\AAPL_NYSE'
✅ Saved → ..\Data\stock_ohlcv\AAPU_NASDAQ.csv
✅ Saved → ..\Data\stock_ohlcv\ABBOTINDIA_NSE.csv
❌ Failed for ABBV (NYSE/NASDAQ): Cannot save file into a non-existent directory: '..\Data\stock_ohlcv\ABBV_NYSE'
✅ Saved → ..\Data\stock_ohlcv\ABCAPITAL_NSE.csv
❌ Failed for ABNB (NYSE/NASDAQ): Cannot save file into a non-existent directory: '..\Data\stock_ohlcv\ABNB_NYSE'
❌ Failed for ABT (NYSE/NASDAQ): Cannot save file into a non-existent directory: '..\Data\stock_ohlcv\ABT_NYSE'
✅ Saved → ..\Data\stock_ohlcv\ALKEM_NSE.csv
❌ Failed for AMGN (NYSE/NASDAQ): Cannot save file into a non